## **Packages to be installed during Runtime**

In [ ]:
pip install anvil-uplink #Helps you connect with the Anvil Server

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp37-none-any.whl size=45216 sha256=d75392150b940b690c0b4f71a5eb3400bf98959886a42359780ba9e932b6be47
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 65kB/s 
     |████████████████████████████████| 204kB 38.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=a90deea964777cd5ef5e8f5b831e8d32f600e31905a8ee99c45f1b75b6860031
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


## **Import Necessary Libraries**

In [ ]:
#To import the Anvil Server
import anvil.server

#For the Spark Session
from pyspark.sql import SparkSession
from pyspark import SparkContext

#For Data manipulation
from pyspark.sql.types import StringType
from pyspark.ml import linalg as ml_linalg
from pyspark.mllib.linalg import Vectors as MLLibVectors

#For NLP
from pyspark.ml.feature import HashingTF,RegexTokenizer,StopWordsRemover
from pyspark.mllib.classification import SVMModel

## **Creating a Session and Loading the files**

In [ ]:
sc = SparkContext(); #Connection to Spark Cluster
spark = SparkSession.builder.appName('Sentiment Analysis').getOrCreate() #Create a Spark Session
anvil.server.connect("UWYF3KZT555UGL6GPFDDCCBP-CZNWQDQQC76FC5MI") #Connect to the Anvil server with unique ID

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [ ]:
!unzip BDA_Endsem_Anvil_Files.zip #Unzips all the files required

Archive:  BDA_Endsem_Anvil_Files.zip
replace vec/metadata/part-00000? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: vec/metadata/part-00000  
replace vec/metadata/_SUCCESS? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: vec/metadata/_SUCCESS   
replace stop_words/metadata/part-00000? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: stop_words/metadata/part-00000  
replace stop_words/metadata/_SUCCESS? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: stop_words/metadata/_SUCCESS  
replace svm_model/data/part-00000-1dba9347-3883-4895-8117-a515e67d221c-c000.snappy.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: svm_model/data/part-00000-1dba9347-3883-4895-8117-a515e67d221c-c000.snappy.parquet  
replace svm_model/data/_SUCCESS? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: svm_model/data/_SUCCESS  
replace svm_model/metadata/part-00000? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: svm_model/metadata/part-00000  
replace svm_model/metadata/_

In [ ]:
#Load the pre-saved SVM Models and all the vectorizers
loaded_tok=RegexTokenizer.load("tokenizer")
loaded_stop=StopWordsRemover.load("stop_words")
loaded_vec=HashingTF.load("vec")
loaded_model=SVMModel.load(sc,"svm_model")

## **Main Functions for Computation**

In [ ]:
#Funtion to convert a Sparse and Dense vector to MLLIB format
def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))

### **Instructions for running the GUI**

[Anvil GUI](https://CZNWQDQQC76FC5MI.anvil.app/CTX52FSLFD4YJVPTIRVSXLSV)


> This is the link that connects you to the Anvil Web.
**Run the next cell first and then enter your reviews**



**Example Positive Review** -> My, that cup helped me so much. I made steaming hot coffee one day and clumsy me dropped the cup,but it was so strong that it didn't break at all.

**Example Negative Review** -> My experience with this watch was abysmal. The day after I bought it, it stopped working and it took me INR 1000 to fix. It worked exactly for one more week and stopped again. Don't buy this and save your money

Manually stop running the next cell after usage


In [ ]:
@anvil.server.callable #To call the Anvil Server

#Function that classifies a review
def classify(document):
  doc_list=[document]
  df=spark.createDataFrame(doc_list,StringType())
  label = {0: 'negative', 1: 'positive'}

  #Tokenizer
  loaded_tok.setInputCol("value")
  loaded_tok.setOutputCol("Words")
  X1=loaded_tok.transform(df)
  
  #Remove Stop Words
  loaded_stop.setInputCol("Words")
  loaded_stop.setOutputCol("Important_words")
  X2=loaded_stop.transform(X1)
  
  #Hashing TF
  loaded_vec.setInputCol("Important_words")
  loaded_vec.setOutputCol("Features")
  X = loaded_vec.transform(X2)
  r = X.rdd.map(lambda d:as_mllib(d['Features']))

  #Predict the label of the review
  y = loaded_model.predict(r)
  y =  y.collect()[0]
  return label[y]

anvil.server.wait_forever() #Server waits until manual termination

KeyboardInterrupt: ignored

## **Thank You!**